<a href="https://colab.research.google.com/github/Rocco000/OncoVision/blob/main/Scripts/ModelsScripts/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the necessary libraries

In [ ]:
!pip install grad-cam
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

Code to obtain the size of the first linear layer in GA2

In [ ]:
def find_last_convolutional_layer(model, layers):
  last = model.layer0
  i = len(layers)

  while i>0:
    match i:
      case 16:
        if isinstance(model.layer16, torch.nn.Conv2d):
          last = model.layer16
          break
      case 15:
        if isinstance(model.layer15, torch.nn.Conv2d):
          last = model.layer15
          break
      case 14:
        if isinstance(model.layer14, torch.nn.Conv2d):
          last = model.layer14
          break
      case 13:
        if isinstance(model.layer13, torch.nn.Conv2d):
          last = model.layer13
          break
      case 12:
        if isinstance(model.layer12, torch.nn.Conv2d):
          last = model.layer12
          break
      case 11:
        if isinstance(model.layer11, torch.nn.Conv2d):
          last = model.layer11
          break
      case 10:
        if isinstance(model.layer10, torch.nn.Conv2d):
          last = model.layer10
          break
      case 9:
        if isinstance(model.layer9, torch.nn.Conv2d):
          last = model.layer9
          break
      case 8:
        if isinstance(model.layer8, torch.nn.Conv2d):
          last = model.layer8
          break
      case 7:
        if isinstance(model.layer7, torch.nn.Conv2d):
          last = model.layer7
          break
      case 6:
        if isinstance(model.layer6, torch.nn.Conv2d):
          last = model.layer6
          break
      case 5:
        if isinstance(model.layer5, torch.nn.Conv2d):
          last = model.layer5
          break
      case 4:
        if isinstance(model.layer4, torch.nn.Conv2d):
          last = model.layer4
          break
      case 3:
        if isinstance(model.layer3, torch.nn.Conv2d):
          last = model.layer3
          break
      case 2:
        if isinstance(model.layer2, torch.nn.Conv2d):
          last = model.layer2
          break
      case 1:
        if isinstance(model.layer1, torch.nn.Conv2d):
          last = model.layer1
          break

    i = i-1

  return last

Function to conduct explainability

In [ ]:
def explain_prediction(model, images, predictions, prediction_type, path_prefix, layers=None):
  #Set the model in evaluation mode
  model.eval()

  #Take the last convolutional layer
  target_layers = None
  if layers is None:
    #GA1 and base model
    target_layers=[model.layer5[0]]
  else:
    #Find the last convolutional layer of GA2 solution
    last_layer = find_last_convolutional_layer(model, layers)
    target_layers=[last_layer]
    print("Last convolutiona layer GA2: ")
    print(target_layers)


  #Construct the CAM object
  cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True if device=="cuda" else False)

  targets = [ClassifierOutputTarget(1)]

  i = 0
  for img in images:
    if prediction_type == 1:
      print("Explainability on ",i+1,"-th image on which the model wrong!")
    elif prediction_type == 2:
      print("Explainability on ",i+1,"-th image")
    else:
      print("Explainability on ",i+1,"-th image on which the model predicts correct!")

    if prediction_type != 2:
      print("Model predicted: ",predictions[i][1],", the true label is: ",predictions[i][0])
    else:
      print("Model predicted: ",predictions[i])

    img = img.unsqueeze(0)
    img = img.to("cuda")

    grayscale_cam = cam(input_tensor=img, targets=targets)
    grayscale_cam = grayscale_cam[0, :]

    img = img.cpu()
    #Convert the tensor in the original image
    rgb_img = img.numpy()[0]

    visualization = show_cam_on_image(rgb_img.transpose(1,2,0), grayscale_cam, use_rgb=False)

    plt.subplot(1, 2, 1)
    plt.imshow(rgb_img.transpose(1,2,0))
    plt.title("Original image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(visualization)
    plt.title("Explainability")
    plt.axis('off')
    if prediction_type == 1:
      plt.savefig(path_prefix+"Wrong"+str(i)+".png")
    elif prediction_type == 2:
      plt.savefig(path_prefix+"Image"+str(i)+".png")
    else:
      plt.savefig(path_prefix+"Correct"+str(i)+".png")
    plt.show()

    i+=1